In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms

### 내장 데이터셋 로드

- `FashionMNIST` 데이터셋 로드하는 예제


- `root`: 데이터셋을 다운로드 받을 경로(폴더) 지정.
- `train`: `True`로 설정된 경우 `train` 데이터셋에서 로드하며, `False`인 경우 `test` 데이터셋에서 로드
- `download`: `True`로 설정된 경우, 인터넷으로부터 데이터셋을 다운로드 받아 지정된 `root` 디렉토리에 다운로드
- `transform`: 이미지 `transform` 적용


In [ ]:
# Image Transform 정의
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

In [ ]:
# train(학습용) 데이터셋 로드
train = datasets.FashionMNIST(root='data',
                              train=True,         # set True
                              download=True,      # 다운로드
                              transform=transform  # transform 적용. (0~1 로 정규화)
                              )

In [ ]:
# test(학습용) 데이터셋 로드
test = datasets.FashionMNIST(root='data',
                             train=False,       # set to False
                             download=True,     # 다운로드
                             transform=transform  # transform 적용. (0~1 로 정규화)
                             )

`FashionMNIST` 데이터셋 시각화

- 총 10개의 카테고리로 구성되어 있으며, `Label`은 아래 코드에서 `labels_map`에 정의되어 있습니다.
- 출처: [zalandoresearch/fashion-mnist](https://github.com/zalandoresearch/fashion-mnist)


In [ ]:
import matplotlib.pyplot as plt

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

figure = plt.figure(figsize=(12, 8))
cols, rows = 8, 5

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train), size=(1,)).item()
    img, label = train[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(torch.permute(img, (1, 2, 0)), cmap="gray")
plt.show()

### torch.utils.data.DataLoader로 데이터셋 로더 구성


In [ ]:
batch_size = 32  # batch_size 지정
num_workers = 8  # Thread 숫자 지정 (병렬 처리에 활용할 쓰레드 숫자 지정)

In [ ]:
train_loader = torch.utils.data.DataLoader(train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=num_workers)

In [ ]:
test_loader = torch.utils.data.DataLoader(test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=num_workers)

## 모델 생성


In [ ]:
# Device 설정 (cuda:0 혹은 cpu)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        # 마지막 출력층의 Neuron은 Class 개수로 설정
        self.output = nn.Linear(32, num_classes)

    def forward(self, x):
        # (B, 1, 28, 28) -> (B, 28*28)
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.output(x)
        return x

In [ ]:
# 모델 생성
model = Net(num_classes=10)

# 모델을 device 에 올립니다. (cuda:0 혹은 cpu)
model.to(device)
model

## 손실함수 / 옵티마이저 정의


In [ ]:
# Mean Squared Error(MSE) 오차 정의
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 설정: model.paramters()와 learning_rate 설정
optimizer = optim.Adam(model.parameters(), lr=0.005)

## 훈련


In [ ]:
# 최대 반복 횟수 정의
num_epoch = 20

# loss 기록하기 위한 list 정의
losses = []
accs = []

# 훈련 모드 활성화
model.train()

for epoch in range(num_epoch):
    # loss 초기화
    running_loss = 0
    # 정확도 계산
    running_acc = 0

    for x, y in train_loader:
        # x, y 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        x = x.to(device)
        y = y.to(device)

        # 그라디언트 초기화 (초기화를 수행하지 않으면 계산된 그라디언트는 누적됩니다.)
        optimizer.zero_grad()

        # output 계산: model의 __call__() 함수 호출
        output = model(x)

        # 손실(loss) 계산 (예측 Prediction, 실제 Ground Truth)
        loss = loss_fn(output, y)

        # 미분 계산
        loss.backward()

        # 경사하강법 계산 및 적용
        optimizer.step()

        # 배치별 loss 를 누적합산 합니다.
        running_loss += loss.item()
        running_acc += (output.argmax(dim=1).eq(y).sum().item() / len(y))

    # 누적합산된 배치별 loss값을 배치의 개수로 나누어 Epoch당 loss를 산출합니다.
    loss = running_loss / len(train_loader)
    losses.append(loss)
    acc = running_acc / len(train_loader)
    accs.append(acc)

    # 매 Epoch의 학습이 끝날때 훈련 결과를 출력합니다.
    print(f'{epoch:03d} loss = {loss:.5f}, accuracy = {acc:.5f}')

## 검증


In [ ]:
# 검증모드 진입
model.eval()

with torch.no_grad():
    # loss 초기화
    running_loss = 0
    # 정확도 계산
    running_acc = 0
    for x, y, in test_loader:
        x, y = x.to(device), y.to(device)

        output = model(x)

        running_loss += loss.item()
        running_acc += (output.argmax(dim=1).eq(y).sum().item() / len(y))

    loss = running_loss / len(test_loader)
    acc = running_acc / len(test_loader)

In [ ]:
# 최종 훈련 결과확인
print(f"Loss: {loss:.5f}, Accuracy: {acc:.5f}")